### A much shorter/ipynb version of train.py for demo sampling shakespeare_char

This file is a refined version from sample.py with shakespeare-char and simpler/smaller config for quick demo integration in jupyter lab, it is the same as following command
```
python sample.py --out_dir=out-shakespeare-char --device=cpu --seed=9999 --num_samples=5
```

In [1]:
import os
import pickle
from contextlib import nullcontext
import torch
import tiktoken
from model import GPTConfig, GPT

In [2]:
# -----------------------------------------------------------------------------
init_from = 'resume' # either 'resume' (from an out_dir) or a gpt2 variant (e.g. 'gpt2-xl')
out_dir = 'out' # ignored if init_from is not 'resume'
start = "\n" # or "<|endoftext|>" or etc. Can also specify a file, use as: "FILE:prompt.txt"
num_samples = 10 # number of samples to draw
max_new_tokens = 500 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
seed = 1337
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
compile = False # use PyTorch 2.0 to compile the model to be faster
# -----------------------------------------------------------------------------

In [3]:
device = 'cpu'
seed = 9999

# shakespeare-char
out_dir = 'out-shakespeare-char'
num_samples = 5

# gpt2
# init_from = 'gpt2'
# start='What is the answer to life, the universe, and everyting?'
# num_samples = 3
# max_new_tokens=100


In [4]:
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

In [5]:
# model
if init_from == 'resume':
    # init from a model saved in a specific directory
    ckpt_path = os.path.join(out_dir, 'ckpt.pt')
    checkpoint = torch.load(ckpt_path, map_location=device)
    gptconf = GPTConfig(**checkpoint['model_args'])
    model = GPT(gptconf)
    state_dict = checkpoint['model']
    unwanted_prefix = '_orig_mod.'
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    model.load_state_dict(state_dict)
elif init_from.startswith('gpt2'):
    # init from a given GPT-2 model
    model = GPT.from_pretrained(init_from, dict(dropout=0.0))

model.eval()
model.to(device)
if compile:
    model = torch.compile(model) # requires PyTorch 2.0 (optional)

number of parameters: 0.80M


In [6]:
# look for the meta pickle in case it is available in the dataset folder
load_meta = False
if init_from == 'resume' and 'config' in checkpoint and 'dataset' in checkpoint['config']: # older checkpoints might not have these...
    meta_path = os.path.join('data', checkpoint['config']['dataset'], 'meta.pkl')
    load_meta = os.path.exists(meta_path)
if load_meta:
    print(f"Loading meta from {meta_path}...")
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    # TODO want to make this more general to arbitrary encoder/decoder schemes
    stoi, itos = meta['stoi'], meta['itos']
    encode = lambda s: [stoi[c] for c in s]
    decode = lambda l: ''.join([itos[i] for i in l])
else:
    # ok let's assume gpt-2 encodings by default
    print("No meta.pkl found, assuming GPT-2 encodings...")
    enc = tiktoken.get_encoding("gpt2")
    encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
    decode = lambda l: enc.decode(l)

Loading meta from data/shakespeare_char/meta.pkl...


In [7]:
# encode the beginning of the prompt
if start.startswith('FILE:'):
    with open(start[5:], 'r', encoding='utf-8') as f:
        start = f.read()
start_ids = encode(start)
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

In [8]:
# run generation
with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
            print(decode(y[0].tolist()))
            print('---------------')


The divat not the wiks foaker him? the
Conercimest, o o I thad thaigh dind dible, oh'd you shat.


CENELANTE:
I shak fater thee shis shave dof sir mun o eveakd:
I to that let iff kneact wer the to prasm is!

RUCAMET:
Wen soy ham ind the thae athe aste; and is,
This oy is anld dron mompeseste in no b'st my on?


Last, und shy and simas, him for twith beith woull chover.


SILONENG:
Ah, baw thy ame for a cand!

WLUSOMER:
ARou, hart mast eke nooof The this trashed, whe smeriads.


Comaverte then li
---------------

Nut.

HACABRIICEN:
WhoulGou awstas forting sood marry heas doflle the tha we cone!
we wous cord ofest prostarst, the of my I mere have the iploe
thou so inceadlis'g ind if theight bust sang.

INTAndur, kis and binow, and het thive nosus,
A what sthat so the prays, in foughim hie shis and of.

INILIUS:
He wellt, fed eive, and, why dinge, thit you,
Af thou thing apthe at. Duy I cheer,

BEnnk in end fur and, my I ind the nonou thim sible
And the lippporetit in wis a of for momode